<center>
    <h1>Estudo de Caso - OLX</h1>
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/28/OLX_Logo.jpg/256px-OLX_Logo.jpg"/>
</center>

In [3]:
from typing import TypeVar

from selenium.webdriver.firefox.webdriver import WebDriver as FirefoxWebDriver
from selenium.webdriver.chrome.webdriver import WebDriver as ChromeWebDriver
from selenium.webdriver import Chrome, Firefox, ChromeOptions, FirefoxOptions
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

WebDriver = TypeVar('WebDriver', FirefoxWebDriver, ChromeWebDriver)

In [4]:
def set_driver(browser: str, headless: bool=True, binary_location: str=None, **kwargs) -> WebDriver:
    '''
    Cria uma instancia Webdriver do Firefox ou Google Chrome.
    '''
    browsers = {'firefox': (Firefox, FirefoxOptions), 'chrome': (Chrome, ChromeOptions)}
    webdriver, options = browsers[browser.lower()]
    options = options()    
    options.headless = headless    
    
    if binary_location is not None:
        options.binary_location = binary_location
    
    return webdriver(options=options, **kwargs)

In [5]:
def parse_anuncio(anuncio):
    url = anuncio.find_element_by_tag_name('a').get_attribute('href')
    profissional = anuncio.find_elements_by_tag_name('span') != []
    return url, profissional

In [6]:
url = 'https://df.olx.com.br/imoveis/aluguel'
driver = set_driver('firefox')
wait = WebDriverWait(driver, 10).until

In [ ]:
driver.get(url)
ID_LISTA_ANUNCIOS = 'main-ad-list'
wait(EC.presence_of_element_located((By.ID, ID_LISTA_ANUNCIOS)))

In [ ]:
lista_anuncios = driver.find_element_by_id(ID_LISTA_ANUNCIOS)
anuncios = lista_anuncios.find_elements_by_tag_name('li')
anuncios = [i for i in anuncios if i.get_attribute('class') != 'item list_native']
qtd_imoveis = len(anuncios)
print('Encontrados {} anuncios'.format(qtd_imoveis))

In [ ]:
clientes = []
for i, anuncio in enumerate(anuncios):    
    url, profissional = parse_anuncio(anuncio)
    if profissional is not True:
        clientes.append(url)
print('Encontrados {} clientes'.format(len(clientes)))

In [ ]:
def pega_dados_anuncio(driver, url):
    driver.get(url)
    driver.switch_to.default_content()
    wait(EC.presence_of_element_located((By.ID, 'ad_title')))
    driver.find_element_by_link_text('... ver número').click()
    telefone = wait(EC.presence_of_element_located((By.ID, 'visible_phone')))
    return telefone

In [ ]:
span_fone = pega_dados_anuncio(driver, clientes[0])
imagem = span_fone.find_element_by_tag_name('img')
url_imagem_fone = imagem.get_attribute('src')

In [ ]:
from IPython.display import Image
import requests

imagem = requests.get(url_imagem_fone).content
Image(imagem)

In [ ]:
with open('fone_cliente.gif', 'wb') as f:
    f.write(imagem)